In [410]:
%%HTML 
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );
        
        var div = document.createElement('DIV');            
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';        
                
    }
    function luc21893_refresh() {                
        if( document.querySelector('.code_cell .input') == null ) {            
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';                        
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }                 
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }            
            window.setTimeout( luc21893_refresh, 1000 )
        }        
    }
    
    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';        
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';        
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }
    
    luc21893_refresh()
</script>

In [1]:
import pickle
import pandas as pd
import os
import numpy as np
from datetime import datetime

### Data Collection

Those data is collected from Estonia Weather service(https://www.ilmateenistus.ee/?lang=en) from two source : 
1. Cloud coverage data (per hour,per day)
2. Hourly weather data (per hour,per day)

Data are collected from 2019-Jan-01 to 2021-Nov-01 which come up with 24,864 individual file, after that, that two data set is join by <b>station,date and hour</b> to form one universal data set for data pre-processing and exploratory data analysis.

Furthermore, sunrise and sunset hour data is capture from https://dateandtime.infodateandtime.info for the period (01 Jan 2021 to 31 Dec 2021). It is used to split the universal data set into two part :

|Set|Description|
|---|-----------|
|Datetime| data from universal set with time2 in the range [1 hrs after sunrise(>=), 1 hrs before sunset(<=) ]|
|Nighttime| data from universal set with time2 in the range [1 hrs before sunset(>), 1hrs after sunrise(<) ]| 

In [ ]:
path='./cloudCoverageData'

#cols= ['Station','I_layer_height','I_layer_amount','II_layer_height','II_layer_amount','III_layer_height','III_layer_amount','IV_layer_height','IV_layer_amount','Vertical visibility_m','Present weather','Visibility_km','time','date']
#print(len(cols))
data=pd.DataFrame()
for x in os.listdir(path):
    print(f'Processing {x}\r',end='')
    f = open(path+'/'+x,'rb')
    filename = x.split('_')
    tmp = pickle.load(f)
    tmp['date']=filename[1]
    tmp['time']=filename[2]
    data=data.append(tmp)

In [3]:
data.columns=['Station','I_layer_height','I_layer_amount','II_layer_height','II_layer_amount','III_layer_height','III_layer_amount','IV_layer_height','IV_layer_amount','Vertical visibility_m','Present weather','Visibility_km','time2','Date']

In [308]:
f=open('./cloudCoverage_all.pickle','wb')
pickle.dump(data,f)

In [ ]:
#cols=['Station','Air temperature (°C)','Relative humidity (%)','Sea level pressure (hPa)','Pressure tendency (hPa)','Wind','Wind','Wind','Cloud amount (palli)','Present weather (sensor)','Present weather (observer)','Precipitation (mm)','Visibility (km)','time','Date']
path='./hourlyData'
#print(len(cols))
i=0
data1=pd.DataFrame()
for x in os.listdir(path):
    print(f' {i} Processing {x}\r',end='')
    f = open(path+'/'+x,'rb')
    filename = x.split('_')
    tmp = pickle.load(f)
    tmp['date']=filename[1]
    tmp['time2']=filename[2]
    data1=data1.append(tmp)
    i+=1frequent count

In [6]:
data1.columns=['Station','Air temperature (°C)','Relative humidity (%)','Sea level pressure (hPa)','Pressure tendency (hPa)','Wind_Direction','Wind_Speed','Wind_gust','Cloud amount (palli)','Present weather (sensor)','Present weather (observer)','Precipitation (mm)','Visibility (km)','time','Date','time2']

In [314]:
f=open('./hourly_all.pickle','wb')
pickle.dump(data1,f)

In [ ]:
dayLen=pd.read_csv('./sunrise_sunset_data.csv')

In [ ]:
#
#f=open('./hourly_cloudCoverage_join_data','rb')
#hourly_cloudCoverage_join_data=pickle.load(f)
hourly_cloudCoverage_join_data=pd.merge(data,data1,on=['Station','Date','time2'])
hourly_cloudCoverage_join_data['Month']=pd.DatetimeIndex(hourly_cloudCoverage_join_data['Date'],dayfirst=True).month
hourly_cloudCoverage_join_data['date2']=pd.DatetimeIndex(hourly_cloudCoverage_join_data['Date'],dayfirst=True).day
hourly_cloudCoverage_join_data=pd.merge(hourly_cloudCoverage_join_data,dayLen,left_on=['Month','date2'],right_on=['Month','date'])
hourly_cloudCoverage_join_data['time2']=hourly_cloudCoverage_join_data['time2'].astype(int)
#f=open('./hourly_cloudCoverage_join_data','wb')
#pickle.dump(hourly_cloudCoverage_join_data,f)
# Split the dataset into datetime and nighttime 
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data[
                                                        (hourly_cloudCoverage_join_data['time2']>=hourly_cloudCoverage_join_data['Sunrise+1']) &                                                                   (hourly_cloudCoverage_join_data['time2']<=hourly_cloudCoverage_join_data['Sunset-1'])]

hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data[
                                                        ~((hourly_cloudCoverage_join_data['time2']>=hourly_cloudCoverage_join_data['Sunrise+1']) &                                                                   (hourly_cloudCoverage_join_data['time2']<=hourly_cloudCoverage_join_data['Sunset-1']))]
#print(hourly_cloudCoverage_join_data.shape)
#print(hourly_cloudCoverage_join_data_daytime.shape)
#print(hourly_cloudCoverage_join_data_nighttime.shape)
#print(hourly_cloudCoverage_join_data_daytime['time2'].unique())
#print(hourly_cloudCoverage_join_data_nighttime['time2'].unique())

### Data Summary

The data set got 24 columns and 
1. 471,960 (universal set) rows
2. 243,124 (Daytime set) rows
3. 101,726 (NightTime set) rows 
4. 127,110 (Non-used set) rows 
 
The following table shows a brief summary on the columns : 

|Column Name|Variable type|Description|Remark|Source|
|-----------|-------------|-----------|------|:------:|
|I_layer_height| numerical| The height of I layer(lowest) from ground (in m)|--|cloud coverage|
|II_layer_height| numerical |The height of II layer(middle) from ground (in m)|--|cloud coverage|
|III_layer_height| numerical |The height of III layer(middle) from ground (in m)|--|cloud coverage|
|IV_layer_height| numerical |The height of IV layer(His) from ground (in m)|--|cloud coverage|
|Vertical visibility|numerical| The visibility measure vertrically from groud (m)|The field got 449791 nan value,remove|cloud coverage|
|Visibility| numerical| The horizontal visibility in KM| --|cloud coverage|
|Air temperature| numerical| Temperature |data correction is need (divide by 10)|hourly data|
|Relative humidity (%)| numerical| Relative humidity in percentage|--|hourly data|
|Sea level pressure| numerical | pressure in hPa|--|hourly data|
|Pressure tendency (hPa)| numerical| The change in pressure with pervious measurement (hPa)|--|hourly data|
|Wind_Direction| numerical| The direction of wind in degree|--|hourly data|
|Wind_Speed| numerical| the velocity of wind in m/s|--|hourly data|
|Wind_gust|numerical| It is the sudden wind increase (m/s)|--|hourly data|
|Precipitation (mm)| numerical |rainfall measurment in mm|--|hourly data|
|Visibility (km)| numerical| The horizontal visibility in KM| duplicate, remove | hourly data|
|Cloud amount (palli)|category| cloud coverage amount |The field got 453125 nan value, will remove|hourly data|
|Station| category | observation location|--|hourly and cloud coverage|
|I_layer_amount| category | measurment in 8 oktas| transform to numerical by i/8|cloud coverage|
|II_layer_amount| category | measurment in 8 oktas|transform to numerical by i/8|cloud coverage|
|III_layer_amount| category | measurment in 8 oktas|transform to numerical by i/8|cloud coverage|
|IV_layer_amount| category | measurment in 8 oktas|transform to numerical by i/8|cloud coverage|
|Present weather| category | Textual description of current weather|remove, can be replace by other columns|cloud coverage| 
|Present weather (sensor)|category | Textual description of current weather|remove, can be replace by other columns|hourly| 
|Present weather (observer)|category | Textual description of current weather|remove, can be replace by other columns|hourly| 
** Cloud coverage measurement reference : https://en.wikipedia.org/wiki/Oktahttps://en.wikipedia.org/wiki/Okta

Here is a list of category value for category variable :

|column name | list of value |
|------------|---------------|
|I_layer_amount | ['0/8' '1/8' '7/8' '2/8' '6/8' '8/8' '4/8' nan '3/8' '5/8' '-' 'standby' 'startup']|
|II_layer_amount| ['0/8' '7/8' '6/8' nan '4/8' '-' '8/8' '5/8' '3/8' '2/8' '1/8'] |
|III_layer_amount | ['0/8' nan '-' '7/8' '3/8' '5/8' '6/8' '2/8' '8/8' '4/8' '1/8']|
|IV_layer_amount | ['0/8' nan '-' '7/8' '6/8' '8/8' '5/8' '4/8' '3/8' '2/8']|
|Present weather | ['clear' nan 'light snow' 'light rain' 'light rain shower' 'mist' 'fog' 'drizzle' 'rain' 'moderate rain' 'precipitation' 'light snow shower' 'heavy rain' 'heavy rain shower' 'moderate rain shower' 'snow' 'moderate snow' 'heavy snow' 'light rain with snow' 'moderate or heavy rain with snow' 'moderate snow shower' 'dizzle or snow grains' 'haze' 'violent rain shower' 'light or moderate precipitation' 'ice pellets' 'freezing rain' 'shower' 'freezing drizzle' 'snow grains' 'heavy snow shower' 'ice crystals']|
|Present weather (sensor) | ['clear' nan 'light snow' 'light rain' 'light rain shower' 'mist' 'fog' 'drizzle' 'rain' 'moderate rain' 'precipitation' 'light snow shower' 'heavy rain' 'heavy rain shower' 'moderate rain shower' 'snow' 'moderate snow' 'heavy snow' 'light rain with snow' 'moderate or heavy rain with snow' 'moderate snow shower' 'dizzle or snow grains' 'haze' 'violent rain shower' 'light or moderate precipitation' 'ice pellets' 'freezing rain' 'shower' 'freezing drizzle' 'snow grains' 'heavy snow shower' 'ice crystals']|

Checking is done for Present weather and Present weather(sensor) , both columns are identical.

### Data Checking (NaN value on Datetime data set)

|column| Nan Count|
|------|:--------:|
|Station|0|
|I_layer_height|71565|
|I_layer_amount|4061|
|II_layer_height|178795|
|II_layer_amount|4066|
|III_layer_height|215624|
|III_layer_amount|4066|
|IV_layer_height|226324|
|IV_layer_amount|4066|
|Vertical visibility_m|223733|
|Present weather|1911|
|Visibility_km|1572|
|Air temperature (°C)|1688|
|Relative humidity (%)|1681|
|Sea level pressure (hPa)|1678|
|Pressure tendency (hPa)|2230|
|Wind_Direction|13882|
|Wind_Speed|13329|
|Wind_gust|13565|
|Cloud amount (palli)|219376|
|Present weather (sensor)|1911|
|Present weather (observer)|219301|
|Precipitation (mm)|1306|
|Visibility (km)|1572|


In [379]:
hourly_cloudCoverage_join_data_daytime.describe()

,I_layer_height,II_layer_height,III_layer_height,IV_layer_height,Vertical visibility_m,Visibility_km,time2,Air temperature (°C),Relative humidity (%),Sea level pressure (hPa),...,Wind_gust,Cloud amount (palli),Precipitation (mm),Visibility (km),time,Month,date2,date,Sunrise+1,Sunset-1
count,181282.000000,54614.000000,14851.000000,3208.000000,9561.000000,241948.000000,243124.000000,241438.000000,241403.000000,241436.000000,...,228752.000000,9815.000000,242069.000000,241948.000000,243124.000000,243124.000000,243124.000000,243124.000000,243124.000000,243124.000000
mean,1190.418023,2131.341231,2314.351895,2219.850374,1132.451836,232.665374,10.417396,53.263020,85.563949,10119.071211,...,67.795823,7.159348,0.685251,232.665374,290.982495,6.279462,15.655752,15.655752,7.735542,17.015474
std,1517.987660,2314.692004,1813.346358,1832.388621,1513.182930,132.668923,8.717384,72.399781,12.150470,109.244558,...,37.850521,3.776635,3.613738,132.668923,232.859035,3.775418,8.816797,8.816797,1.617282,2.404136
min,30.000000,60.000000,150.000000,240.000000,20.000000,1.000000,0.000000,-301.000000,19.000000,9657.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,5.000000,14.000000
25%,250.000000,840.000000,840.000000,780.000000,70.000000,160.000000,3.000000,7.000000,80.000000,10055.000000,...,39.000000,4.000000,0.000000,160.000000,75.000000,3.000000,8.000000,8.000000,7.000000,15.000000
50%,570.000000,1550.000000,1830.000000,1500.000000,650.000000,200.000000,6.000000,44.000000,89.000000,10131.000000,...,62.000000,10.000000,0.000000,200.000000,220.000000,6.000000,16.000000,16.000000,8.000000,17.000000
75%,1450.000000,2930.000000,3400.000000,3230.000000,1480.000000,350.000000,20.000000,107.000000,95.000000,10193.000000,...,90.000000,10.000000,0.000000,350.000000,478.000000,10.000000,23.000000,23.000000,9.000000,19.000000
max,24300.000000,208100.000000,7580.000000,7500.000000,7230.000000,500.000000,23.000000,291.000000,100.000000,10434.000000,...,335.000000,10.000000,250.000000,500.000000,764.000000,12.000000,31.000000,31.000000,10.000000,21.000000


### Data Clearing , Transformation and adjust (Applied to Daytime and Night Time data)


#### Row's value update according to other's indicator for retain data
1. It is find that some of the NaN that appear in layer_height columns is come with lay_amount = 0/8 , which mean it should be 0, update those Nan to 0
2. For wind direction, Wind_Speed and Wind_gust in nan, it is interpurrted as no wind, with windows direction set to -1 indicate there is no wind direction , Wind_Speed and Wind_guest to 0
3. It is found that most of the nan value for cloud height and amount with Present weather sensor type is clear , update those to 0/8 and 0

#### Rows are dropped due to NaN value that is impossible to fix 
1. Pressure tendency (hPa)' , 'Present weather (sensor)','Precipitation (mm)','Relative humidity (%)','Air temperature (°C) are dropped 
2. Still after those action on layer_amount and layer_height is corrected , there are 2619 rows that is NaN , dropped.

#### Data Transformation/ adjustment 
1. Air Temp is adjusted by divided by 10
2. It is found that those I_layer_amount with value '-' , 'standby' , 'startup' are with Present weather (sensor) = clear, update those to 0/8
3. It is also found that II_layer_amount,III_layer_amount,IV_layer_amount with value '-' is a subset above , update to 0/8
4. Those layer_amount value is changed to numerical form by x/8

After the Data cleaning, those data set size reduce to 19 columns with rows : 

|Data Set|Before|After|$\delta$|
|--------|------|-----|-----|
|Daytime|243,124|235,994|7130|
|NightTime|101,726|99,187|2539|

### Code for Daytime data set

In [ ]:
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['I_layer_amount'].isna()== False]
#It is find that some of the nan that appear in layer_height columns is come with lay_amount = 0/8 , which mean it should be 0 
hourly_cloudCoverage_join_data_daytime.loc[hourly_cloudCoverage_join_data_daytime['I_layer_amount']=='0/8', 'I_layer_height']=0
hourly_cloudCoverage_join_data_daytime.loc[hourly_cloudCoverage_join_data_daytime['II_layer_amount']=='0/8', 'II_layer_height']=0
hourly_cloudCoverage_join_data_daytime.loc[hourly_cloudCoverage_join_data_daytime['III_layer_amount']=='0/8', 'III_layer_height']=0
hourly_cloudCoverage_join_data_daytime.loc[hourly_cloudCoverage_join_data_daytime['IV_layer_amount']=='0/8', 'IV_layer_height']=0

# Those wind direction, Wind_Speed and Wind_gust in nan is interpurrted as no wind , with windows direction set to -1 indicate there is no wind direction.
hourly_cloudCoverage_join_data_daytime['Wind_Speed']=hourly_cloudCoverage_join_data_daytime['Wind_Speed'].fillna(0)
hourly_cloudCoverage_join_data_daytime['Wind_gust']=hourly_cloudCoverage_join_data_daytime['Wind_gust'].fillna(0)
hourly_cloudCoverage_join_data_daytime['Wind_Direction']=hourly_cloudCoverage_join_data_daytime['Wind_Direction'].fillna(-1)
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime.drop(['Vertical visibility_m','Visibility (km)','Cloud amount (palli)','Present weather','Present weather (observer)','Present weather (observer)'],axis=1)
# drop all rows that 'Pressure tendency (hPa)' , 'Present weather (sensor)','Precipitation (mm)','Relative humidity (%)','Air temperature (°C)' is nan , not possible to fix 
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['Pressure tendency (hPa)'].isna()==False]
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['Present weather (sensor)'].isna()==False]
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['Precipitation (mm)'].isna()==False]
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['Relative humidity (%)'].isna()==False]
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[hourly_cloudCoverage_join_data_daytime['Air temperature (°C)'].isna()==False]

In [394]:
# It is found that most of the nan value for cloud height and amount with Present weather sensor type is clear , update those to 0/8 and 0
hourly_cloudCoverage_join_data_daytime[(
                                        (hourly_cloudCoverage_join_data_daytime['I_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['II_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['III_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['IV_layer_height'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_daytime['Present weather (sensor)']=='clear')
                                      ]
hourly_cloudCoverage_join_data_daytime.loc[(
                                        (hourly_cloudCoverage_join_data_daytime['I_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['II_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['III_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['IV_layer_amount'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_daytime['Present weather (sensor)']=='clear')
                                      ,['I_layer_amount','II_layer_amount','III_layer_amount','IV_layer_amount']]='0/8'

hourly_cloudCoverage_join_data_daytime.loc[(
                                        (hourly_cloudCoverage_join_data_daytime['I_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['II_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['III_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_daytime['IV_layer_height'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_daytime['Present weather (sensor)']=='clear')
                                      ,['I_layer_height','II_layer_height','III_layer_height','IV_layer_height']]=0

In [396]:
# There are 2619 row that are unable to fix , drop
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[(
                                        (hourly_cloudCoverage_join_data_daytime['I_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_daytime['II_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_daytime['III_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_daytime['IV_layer_height'].isna()==False) ) ]
# There are 3 row that are unable to fix , drop
hourly_cloudCoverage_join_data_daytime=hourly_cloudCoverage_join_data_daytime[
                                        (hourly_cloudCoverage_join_data_daytime['Visibility_km'].isna()==False)]

In [397]:
# Data Transformation/ adjustment 
# It is found that those I_layer_amount with value '-' , 'standby' , 'startup' are with Present weather (sensor) = clear, update those to 0/8
# It is also found that II_layer_amount,III_layer_amount,IV_layer_amount with value '-' is a subset above , update to 0/8 
hourly_cloudCoverage_join_data_daytime.loc[(hourly_cloudCoverage_join_data_daytime['I_layer_amount']=='-') |
(hourly_cloudCoverage_join_data_daytime['I_layer_amount']=='standby') |
(hourly_cloudCoverage_join_data_daytime['I_layer_amount']=='startup' ),'I_layer_amount']='0/8'

hourly_cloudCoverage_join_data_daytime.loc[(hourly_cloudCoverage_join_data_daytime['II_layer_amount']=='-'),'II_layer_amount']='0/8'
hourly_cloudCoverage_join_data_daytime.loc[(hourly_cloudCoverage_join_data_daytime['III_layer_amount']=='-'),'III_layer_amount']='0/8'
hourly_cloudCoverage_join_data_daytime.loc[(hourly_cloudCoverage_join_data_daytime['IV_layer_amount']=='-'),'IV_layer_amount']='0/8'



#hourly_cloudCoverage_join_data_daytime['Air temperature (°C)']=hourly_cloudCoverage_join_data_daytime['Air temperature (°C)']/10
hourly_cloudCoverage_join_data_daytime['I_layer_amount']=hourly_cloudCoverage_join_data_daytime['I_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_daytime['II_layer_amount']=hourly_cloudCoverage_join_data_daytime['II_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_daytime['III_layer_amount']=hourly_cloudCoverage_join_data_daytime['III_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_daytime['IV_layer_amount']=hourly_cloudCoverage_join_data_daytime['IV_layer_amount'].str.extract('(\d+)').astype(int)/8

In [ ]:

hourly_cloudCoverage_join_data_daytime.describe()

### Code for Night Time data set

In [401]:
#hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['I_layer_amount'].isna()== False]
#It is find that some of the nan that appear in layer_height columns is come with lay_amount = 0/8 , which mean it should be 0 
hourly_cloudCoverage_join_data_nighttime.loc[hourly_cloudCoverage_join_data_nighttime['I_layer_amount']=='0/8', 'I_layer_height']=0
hourly_cloudCoverage_join_data_nighttime.loc[hourly_cloudCoverage_join_data_nighttime['II_layer_amount']=='0/8', 'II_layer_height']=0
hourly_cloudCoverage_join_data_nighttime.loc[hourly_cloudCoverage_join_data_nighttime['III_layer_amount']=='0/8', 'III_layer_height']=0
hourly_cloudCoverage_join_data_nighttime.loc[hourly_cloudCoverage_join_data_nighttime['IV_layer_amount']=='0/8', 'IV_layer_height']=0

# Those wind direction, Wind_Speed and Wind_gust in nan is interpurrted as no wind , with windows direction set to -1 indicate there is no wind direction.
hourly_cloudCoverage_join_data_nighttime['Wind_Speed']=hourly_cloudCoverage_join_data_nighttime['Wind_Speed'].fillna(0)
hourly_cloudCoverage_join_data_nighttime['Wind_gust']=hourly_cloudCoverage_join_data_nighttime['Wind_gust'].fillna(0)
hourly_cloudCoverage_join_data_nighttime['Wind_Direction']=hourly_cloudCoverage_join_data_nighttime['Wind_Direction'].fillna(-1)
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime.drop(['Vertical visibility_m','Visibility (km)','Cloud amount (palli)','Present weather','Present weather (observer)','Present weather (observer)'],axis=1)
# drop all rows that 'Pressure tendency (hPa)' , 'Present weather (sensor)','Precipitation (mm)','Relative humidity (%)','Air temperature (°C)' is nan , not possible to fix 
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['Pressure tendency (hPa)'].isna()==False]
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['Present weather (sensor)'].isna()==False]
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['Precipitation (mm)'].isna()==False]
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['Relative humidity (%)'].isna()==False]
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[hourly_cloudCoverage_join_data_nighttime['Air temperature (°C)'].isna()==False]

In [403]:
# It is found that most of the nan value for cloud height and amount with Present weather sensor type is clear , update those to 0/8 and 0
hourly_cloudCoverage_join_data_nighttime[(
                                        (hourly_cloudCoverage_join_data_nighttime['I_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['II_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['III_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['IV_layer_height'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_nighttime['Present weather (sensor)']=='clear')
                                      ]
hourly_cloudCoverage_join_data_nighttime.loc[(
                                        (hourly_cloudCoverage_join_data_nighttime['I_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['II_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['III_layer_amount'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['IV_layer_amount'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_nighttime['Present weather (sensor)']=='clear')
                                      ,['I_layer_amount','II_layer_amount','III_layer_amount','IV_layer_amount']]='0/8'

hourly_cloudCoverage_join_data_nighttime.loc[(
                                        (hourly_cloudCoverage_join_data_nighttime['I_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['II_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['III_layer_height'].isna()==True) |
                                        (hourly_cloudCoverage_join_data_nighttime['IV_layer_height'].isna()==True) ) 
                                        & (hourly_cloudCoverage_join_data_nighttime['Present weather (sensor)']=='clear')
                                      ,['I_layer_height','II_layer_height','III_layer_height','IV_layer_height']]=0

In [404]:
# There are 2619 row that are unable to fix , drop
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[(
                                        (hourly_cloudCoverage_join_data_nighttime['I_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_nighttime['II_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_nighttime['III_layer_height'].isna()==False) &
                                        (hourly_cloudCoverage_join_data_nighttime['IV_layer_height'].isna()==False) ) ]
# There are 3 row that are unable to fix , drop
hourly_cloudCoverage_join_data_nighttime=hourly_cloudCoverage_join_data_nighttime[
                                        (hourly_cloudCoverage_join_data_nighttime['Visibility_km'].isna()==False)]


In [405]:
# Data Transformation/ adjustment 
# It is found that those I_layer_amount with value '-' , 'standby' , 'startup' are with Present weather (sensor) = clear, update those to 0/8
# It is also found that II_layer_amount,III_layer_amount,IV_layer_amount with value '-' is a subset above , update to 0/8 
hourly_cloudCoverage_join_data_nighttime.loc[(hourly_cloudCoverage_join_data_nighttime['I_layer_amount']=='-') |
(hourly_cloudCoverage_join_data_nighttime['I_layer_amount']=='standby') |
(hourly_cloudCoverage_join_data_nighttime['I_layer_amount']=='startup' ),'I_layer_amount']='0/8'

hourly_cloudCoverage_join_data_nighttime.loc[(hourly_cloudCoverage_join_data_nighttime['II_layer_amount']=='-'),'II_layer_amount']='0/8'
hourly_cloudCoverage_join_data_nighttime.loc[(hourly_cloudCoverage_join_data_nighttime['III_layer_amount']=='-'),'III_layer_amount']='0/8'
hourly_cloudCoverage_join_data_nighttime.loc[(hourly_cloudCoverage_join_data_nighttime['IV_layer_amount']=='-'),'IV_layer_amount']='0/8'



#hourly_cloudCoverage_join_data_nighttime['Air temperature (°C)']=hourly_cloudCoverage_join_data_nighttime['Air temperature (°C)']/10
hourly_cloudCoverage_join_data_nighttime['I_layer_amount']=hourly_cloudCoverage_join_data_nighttime['I_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_nighttime['II_layer_amount']=hourly_cloudCoverage_join_data_nighttime['II_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_nighttime['III_layer_amount']=hourly_cloudCoverage_join_data_nighttime['III_layer_amount'].str.extract('(\d+)').astype(int)/8
hourly_cloudCoverage_join_data_nighttime['IV_layer_amount']=hourly_cloudCoverage_join_data_nighttime['IV_layer_amount'].str.extract('(\d+)').astype(int)/8

In [411]:
f=open('./hourly_cloudCoverage_join_data_daytime_clean','wb')
pickle.dump(hourly_cloudCoverage_join_data_daytime,f)
f=open('./hourly_cloudCoverage_join_data_nighttime_clean','wb')
pickle.dump(hourly_cloudCoverage_join_data_nighttime,f)